test the build-in model

In [1]:
#import useful libraries
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from tensorflow import keras
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from skimage import io
import tensorflow as tf
import pims
import matplotlib.patches as mpatches
from skimage.color import rgb2gray
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from functions import *

In [3]:
BATCH_SIZE = 10 # the higher the better
IMG_WIDTH = 128 # for faster computing on kaggle
IMG_HEIGHT = 128 # for faster computing on kaggle
IMG_CHANNELS = 3 
TEST_PATH='./test_set/'# our project
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
n_num=num(dirname='tifs - unanalysed/')
#build our test set
test_ids = next(os.walk(TEST_PATH))[1]
X_test = np.zeros((n_num, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
#just to ensure that len of X_test is equal to len of sizes_test
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
n=0
for id_ in tqdm(test_ids, total=len(test_ids)):
    path = TEST_PATH + id_+'/images/'
    dir_3=os.listdir(path)
    k=len(dir_3)
    for i in range(0,k): 
        img = imread(path +dir_3[i])[:,:,:IMG_CHANNELS] # look for all pictures with name Series001
        sizes_test.append([img.shape[0], img.shape[1]])# append sizes of figures i.e. 256*256 pixel: [256,256]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_test[n]=img
        n+=1
print('Done!')

Getting and resizing test images ... 


100%|██████████| 30/30 [00:04<00:00,  7.30it/s]

Done!


In [5]:
for id_ in tqdm(test_ids, total=len(test_ids)):
    path = TEST_PATH + id_+'/images/'
    dir_3=os.listdir(path)
    k=len(dir_3)
    for i in range(0,k):   
        image = imread(path +dir_3[i])[:,:,:3]# load the pictures
        image = rgb2gray(image) #convert to greyscale after loading
        thresh = threshold_otsu(image) # perform automatic thresholding
        bw = closing(image >thresh,square(3)) #convert 0，1 to bool
        # remove artifacts connected to image border
        cleared = clear_border(bw)
        # label image regions
        label_image = label(cleared)
        # to make the background transparent, pass the value of `bg_label`,
        # and leave `bg_color` as `None` and `kind` as `overlay`
        image_label_overlay = label2rgb(label_image, image=image, bg_label=0)
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.imshow(image_label_overlay)
        for region in regionprops(label_image):
            # take regions with large enough areas
            if region.area >= 150:
                # draw rectangle around segmented coins
                minr, minc, maxr, maxc = region.bbox
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='red', linewidth=2)
                ax.add_patch(rect)
        ax.set_axis_off()
        plt.tight_layout()
        figure_save_path ='label_set/'+id_
        if not os.path.exists(figure_save_path):# if directory not exist, create it
            os.makedirs(figure_save_path)   
        plt.savefig(os.path.join(figure_save_path , '{}'.format(dir_3[i])),bbox_inches='tight', pad_inches = -0.1)
        plt.close()
                        

100%|██████████| 30/30 [00:28<00:00,  1.04it/s]


In [24]:
#load the model and use it to make predicted results
model = load_model('model.h5', custom_objects={'mean_iou': tf.keras.metrics.MeanIoU(num_classes=2)})
preds_test = model.predict(X_test, verbose=1)

#Threshold predictions
preds_test_t = (preds_test > 0.8).astype(np.uint8)

#Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

6/6 [==============================] - 2s 297ms/step


In [33]:
ID=0
ID_dir={}
for id_ in test_ids:
    path = TEST_PATH + id_+'/images/'
    dir_3=os.listdir(path)
    k=len(dir_3)
    for i in range(0,k): 
        ID_dir[ID]=dir_3[i]
        ID+=1
print(ID_dir)        

{0: '1.2%_1Hour_A-Tubulin.lif - Series001.tif', 1: '1.2%_1Hour_A-Tubulin.lif - Series002.tif', 2: '1.2%_1Hour_A-Tubulin.lif - Series003.tif', 3: '1.2%_1Hour_A-Tubulin.lif - Series004.tif', 4: '1.2%_1Hour_A-Tubulin.lif - Series005.tif', 5: '1.2%_1Hour_A-Tubulin.lif - Series006.tif', 6: '1.2%_1Hour_B-Actin.lif - Series001.tif', 7: '1.2%_1Hour_B-Actin.lif - Series002.tif', 8: '1.2%_1Hour_B-Actin.lif - Series003.tif', 9: '1.2%_1Hour_B-Actin.lif - Series004.tif', 10: '1.2%_1Hour_B-Actin.lif - Series005.tif', 11: '1.2%_1Hour_B-Actin.lif - Series006.tif', 12: '1.2%_1Hour_Caveolin 1.lif - Series001.tif', 13: '1.2%_1Hour_Caveolin 1.lif - Series002.tif', 14: '1.2%_1Hour_Caveolin 1.lif - Series003.tif', 15: '1.2%_1Hour_Caveolin 1.lif - Series004.tif', 16: '1.2%_1Hour_Caveolin 1.lif - Series005.tif', 17: '1.2%_1Hour_Caveolin 1.lif - Series006.tif', 18: '1.2%_1Hour_Clathrin.lif - Series001.tif', 19: '1.2%_1Hour_Clathrin.lif - Series002.tif', 20: '1.2%_1Hour_Clathrin.lif - Series003.tif', 21: '1.2%_

Getting and resizing test images ... 


100%|██████████| 30/30 [00:04<00:00,  7.30it/s]

Done!


In [41]:
# Perform a sanity check on some random training samples
for i in range(0,len(X_test)):
    plt.figure(figsize=(12,12))
    plt.subplot(121)
    plt.imshow(X_test[i])
    #plt.title('original test')
    plt.axis('off')
    plt.subplot(122)
    plt.imshow(preds_test_t[i])
    #plt.title('Predictions')
    plt.axis('off')
    #plt.subplot(313)
    #plt.imshow(np.squeeze(preds_test_t[i]))
    #plt.plt.title('Threshold predictions')
    #plt.axis('off')
    plt.tight_layout()
    figure_save_path ='pred_set/'
    if not os.path.exists(figure_save_path):# if directory not exist, create it
        os.makedirs(figure_save_path)   
    plt.savefig(os.path.join(figure_save_path , '{}'.format(ID_dir[i])),bbox_inches='tight', pad_inches = -0.1)
    plt.close()    

In [ ]:
# Generate an initial image with two overlapping circles
x, y = np.indices((80, 80))
x1, y1, x2, y2 = 28, 28, 44, 52
r1, r2 = 16, 20
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
image = np.logical_or(mask_circle1, mask_circle2)

# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=image)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)
labels = watershed(-distance, markers, mask=image)
fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()